In [1]:
using CompTime, InteractiveUtils, MacroTools

In [2]:
function pw_log_rec(n, x)
    if iszero(n)
        one(x)
    elseif isone(n)
        x
    elseif isodd(n)
        x * pw_log_rec(n - 1, x)
    else
        pw_log_rec(n ÷ 2, x * x)
    end
end

pw_log_rec (generic function with 1 method)

In [3]:
Tuple(pw_log_rec(n, "Abc") for n in 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [4]:
function pw_log_loop(n, x)
    if iszero(n)
        return one(x)
    else
        r = one(x)

        while n > 1
            if isodd(n)
                r = r * x
                n = n - 1
            else
                x = x * x
                n = n ÷ 2
            end
        end

        return r * x
    end
end

pw_log_loop (generic function with 1 method)

In [5]:
Tuple(pw_log_loop(n, "Abc") for n = 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [6]:
@ct_enable function pw_log_ct(@ct(n), x)
    @ct(k = n)

    @ct_ctrl if iszero(n)
        return one(x)
    else
        r = one(x)

        @ct_ctrl while k > 1
            @ct_ctrl if isodd(k)
                r = r * x
                @ct(k = k - 1)
            else
                x = x * x
                @ct(k = k ÷ 2)
            end
        end

        return r * x
    end
end

runtime (generic function with 1 method)

In [7]:
Tuple(pw_log_ct(Val{n}, "Abc") for n = 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [8]:
Tuple(runtime(pw_log_ct, n, "Abc") for n = 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [9]:
Tuple(comptime(pw_log_ct, Val{n}, "Abc") for n = 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [10]:
@code_typed pw_log_ct(Val{5}, 10)

CodeInfo(
1 ─ %1 = Base.mul_int(1, x)::Int64
│   %2 = Base.mul_int(x, x)::Int64
│   %3 = Base.mul_int(%2, %2)::Int64
│   %4 = Base.mul_int(%1, %3)::Int64
└──      return %4
) => Int64

In [11]:
debug(pw_log_ct, Val{5}, 10) |> Base.remove_linenums! |> MacroTools.flatten

quote
    5
    r = one(x)
    r = r * x
    4
    x = x * x
    2
    x = x * x
    1
    return r * x
end

In [12]:
@ct_enable function pw_log_opt_ct(@ct(n), x)
    @ct(k = n)

    @ct_ctrl if iszero(k)
        return one(x)
    else
        @ct(r1 = true)

        @ct_ctrl while k > 1
            @ct_ctrl if isodd(k)
                @ct_ctrl if r1
                    r = x
                    @ct(r1 = false)
                else
                    r = r * x
                end
                @ct(k = k - 1)
            else
                x = x * x
                @ct(k = k ÷ 2)
            end
        end

        @ct_ctrl if r1
            return x
        else
            return r * x
        end
    end
end

runtime (generic function with 2 methods)

In [13]:
Tuple(pw_log_opt_ct(Val{n}, "Abc") for n in 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [14]:
@code_typed pw_log_opt_ct(Val{5}, 10)

CodeInfo(
1 ─ %1 = Base.mul_int(x, x)::Int64
│   %2 = Base.mul_int(%1, %1)::Int64
│   %3 = Base.mul_int(x, %2)::Int64
└──      return %3
) => Int64

In [15]:
debug(pw_log_opt_ct, Val{0}, 10) |> Base.remove_linenums! |> MacroTools.flatten

quote
    0
    return one(x)
end

In [16]:
debug(pw_log_opt_ct, Val{1}, 10) |> Base.remove_linenums! |> MacroTools.flatten

quote
    1
    true
    return x
end

In [17]:
debug(pw_log_opt_ct, Val{4}, 10) |> Base.remove_linenums! |> MacroTools.flatten

quote
    4
    true
    x = x * x
    2
    x = x * x
    1
    return x
end

In [18]:
debug(pw_log_opt_ct, Val{5}, 10) |> Base.remove_linenums! |> MacroTools.flatten

quote
    5
    true
    r = x
    false
    4
    x = x * x
    2
    x = x * x
    1
    return r * x
end

In [19]:
debug(pw_log_opt_ct, Val{7}, 10) |> Base.remove_linenums! |> MacroTools.flatten

quote
    7
    true
    r = x
    false
    6
    x = x * x
    3
    r = r * x
    2
    x = x * x
    1
    return r * x
end